# Challenge 
Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are ***tasked with predicting their daily sales for up to six weeks in advance***. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

In their first Kaggle competition, Rossmann is challenging you to predict 6 weeks of daily sales for 1,115 stores located across Germany. Reliable sales forecasts enable store managers to create effective staff schedules that increase productivity and motivation. By helping Rossmann create a robust prediction model, you will help store managers stay focused on what’s most important to them: their customers and their teams! 

# Predict Store Sales 
***What we learn from this notebook***

1. **Data Collection and Preparation**
   * Gather Data
   * Remove **TOTALLY** unrelated columns from dataset
   * Encode cateorical features
   * Handle too many unique values in categorical features (Cordinality)
   * Missing value handling 
  
  
2. **Exploratory Data Analysis (EDA)**
   * Remove outliers
   * Understand data distribution (mean, median, mode, standard deviation).
   * Check for correlations & covariance between features.
   
   
3. **Feature Selection & Engineering**
   *  Select relevant features based on correlation, mutual information, feature importance, Lasso, RFE, R-squared, p-values, and VIF.
   * Create new features if necessary.
   
   
4. **Check for Assumptions of Linear Regression**
   * **Linearity**: The relationship between predictors and the target should be linear.
   * **Independence**: Observations should be independent of each other,
   * **Homoscedasticity**: Constant variance of the residuals
   * **Normality of Residuals**: Residuals should be normally distributed.
   * **No Multicollinearity**: Predictors should not be highly correlated with each other.
   
   
5. **Model Training**
   * Split data into training and testing sets.
   * Train the multiple linear regression model.
   
   
6. **Model Evaluation**
   * Evaluate the model using metrics like R-squared, adjusted R-squared, Mean Squared Error (MSE), and Root Mean Squared Error (RMSE).
   * Check residual plots to ensure assumptions are met.
   
   
7. **Predict Sales based on new model created**
 

## 1. Data Collection and Preparation

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        full_path = os.path.join(dirname, filename)
        print (full_path)
        if "store.csv" in full_path:
            df_store = pd.read_csv(full_path)
        elif "train.csv" in full_path:
            df_train = pd.read_csv(full_path)
        elif "test.csv" in full_path:
            df_test = pd.read_csv(full_path)
                                 
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rossmann-store-sales-dataset/store.csv
/kaggle/input/rossmann-store-sales-dataset/train.csv


/tmp/ipykernel_32/1702920036.py:19: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(full_path)


/kaggle/input/rossmann-store-sales-dataset/test.csv


### Data fields
Most of the fields are self-explanatory. The following are descriptions for those that aren't.

**Id** - an Id that represents a (Store, Date) duple within the test set

**Store** - a unique Id for each store

**Sales** - the turnover for any given day (this is what you are predicting)

**Customers** - the number of customers on a given day

**Open** - an indicator for whether the store was open: 0 = closed, 1 = open

**StateHoliday** - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None

**SchoolHoliday** - indicates if the (Store, Date) was affected by the closure of public schools

**StoreType** - differentiates between 4 different store models: a, b, c, d

**Assortment** - describes an assortment level: a = basic, b = extra, c = extended

**CompetitionDistance** - distance in meters to the nearest competitor store

**CompetitionOpenSince[Month/Year]** - gives the approximate year and month of the time the nearest competitor was opened

**Promo** - indicates whether a store is running a promo on that day

**Promo2** - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating

**Promo2Since[Year/Week]** - describes the year and calendar week when the store started participating in Promo2

**PromoInterval** - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store


In [2]:
df_store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [3]:
df_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


#### Check the SHAPE of each DataFrame
#### Merge the train and store data based on the store id to have all associated feature of store in single row

In [4]:
dfs =[df_store,df_train]
for df in dfs:
    #print (df)
    print (df.shape)

(1115, 10)
(1017209, 9)


In [5]:
df = df_train.merge (df_store, how="left", on="Store")
print (df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Date                       1017209 non-null  object 
 3   Sales                      1017209 non-null  int64  
 4   Customers                  1017209 non-null  int64  
 5   Open                       1017209 non-null  int64  
 6   Promo                      1017209 non-null  int64  
 7   StateHoliday               1017209 non-null  object 
 8   SchoolHoliday              1017209 non-null  int64  
 9   StoreType                  1017209 non-null  object 
 10  Assortment                 1017209 non-null  object 
 11  CompetitionDistance        1014567 non-null  float64
 12  CompetitionOpenSinceMonth  693861 non-null   float64
 13  CompetitionO

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


#### Intuitive Approach In Prediction
##### 1.  How *Promo, stateHoliday, SchoolHoliday, Promo2, Promo2SinceWeek* and *Promo2SinceYear* impacted the sales in a store?
  * Whats impact of question#1 in a year, month and weeks. 
      * 2 What's the impact of season in a year impacted the sales in a store?
      * 1.3 What's the impact of the weekend in a year impacted sales in a store?
      * 1.4 How Open on weekend, weekday StateHoliday, SchooldHliday impacted the sales in Store?
      * 1.5 Was there a impact on sales in stores for a month in a given year?
##### 2.  How much sales impacted in a store becuase of these *CompetitionOpenSinceMonth* and *CompetitionOpenSinceYear*?
##### 3.  How much *PromoInterval* impacted the sales in a store?
##### 4. 

### Fix The Data Types 

In [6]:
df['Date'].isnull().sum()

0

In [7]:
df['Date'] = pd.to_datetime(df['Date'])

In [8]:
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


## Missing Values Handling

In [9]:
df.isnull().sum()

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
StoreType                         0
Assortment                        0
CompetitionDistance            2642
CompetitionOpenSinceMonth    323348
CompetitionOpenSinceYear     323348
Promo2                            0
Promo2SinceWeek              508031
Promo2SinceYear              508031
PromoInterval                508031
dtype: int64

### Handling missing value for the feature CompetitionDistance

In [10]:
df[df['CompetitionDistance'].isnull()].head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
290,291,5,2015-07-31,10052,973,1,1,0,1,d,a,NaN,NaN,NaN,0,NaN,NaN,NaN
621,622,5,2015-07-31,6306,540,1,1,0,1,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
878,879,5,2015-07-31,6000,466,1,1,0,0,d,a,NaN,NaN,NaN,1,5.0,2013.0,"Feb,May,Aug,Nov"
1405,291,4,2015-07-30,9030,883,1,1,0,1,d,a,NaN,NaN,NaN,0,NaN,NaN,NaN
1736,622,4,2015-07-30,5412,406,1,1,0,1,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN


#### Checking if there is a relation between other features with the CompetitionDistance

In [11]:
print (df['CompetitionDistance'].isnull().sum() )
print (df['CompetitionOpenSinceMonth'].isnull().sum() )
print (df['CompetitionOpenSinceYear'].isnull().sum() )

2642
323348
323348


#### Looks there is close relation between these features 

In [12]:
df[df['CompetitionDistance'].isna() & df['CompetitionOpenSinceMonth'].isna() & df['CompetitionOpenSinceYear'].isna()].shape

(2642, 18)

#### Lets try to find if there are any Store has its distance already present but only missed at Ramdon 

In [13]:
# Group by 'Store' and check for both NaN and non-NaN values in 'CompetitionDistance'
store_groups = df.groupby('Store')['CompetitionDistance'].agg(
    missing_count = lambda x: x.isna().sum(),
    non_missing_count = lambda x: x.notna().sum()
).reset_index()

# Filter stores that have both missing and non-missing values
stores_with_both = store_groups[(store_groups['missing_count'] > 0) & (store_groups['non_missing_count'] > 0)]

# Display the result
print("Stores with both missing and non-missing CompetitionDistance")
stores_with_both

Stores with both missing and non-missing CompetitionDistance


,Store,missing_count,non_missing_count


No such store where it has value in CompetitionDistance and null at other places CompetitionDistance

#### Lets find the list Stores have the null value in all of these 3 features 

In [14]:
nan_df = df[['Store','CompetitionDistance','CompetitionOpenSinceMonth','CompetitionOpenSinceYear']]

In [15]:
nan_df.groupby('Store')['CompetitionDistance'].size()

Store
1       942
2       942
3       942
4       942
5       942
       ... 
1111    942
1112    942
1113    942
1114    942
1115    942
Name: CompetitionDistance, Length: 1115, dtype: int64

### From above its clearn that only 3 Stores have missing value in all these 3 features. 
**Row percentage of these 3 stores contributes only 0.25% of total.**

**This is an *MISSING at RANdom (MAR)* patttern for these 3 stores.**

**Based on the StoreType and Assortment of other stores, I assume these 3 stores will also have same distance as compitator** 

In [16]:
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [17]:
stores_to_exclude = [291, 622, 879]
store_type_291=['d']
assortment_291=['a']
mask_291 = df[(~df['Store'].isin(stores_to_exclude)) & (df['StoreType'].isin(store_type_291)) & (df['Assortment'].isin(assortment_291))]

In [23]:

stores_to_exclude = [622, 879]
store_type_622=['a']
assortment_622=['c']
mask_622 = df[(~df['Store'].isin(stores_to_exclude)) & (df['StoreType'].isin(store_type_622)) & (df['Assortment'].isin(assortment_622))]

In [29]:

stores_to_exclude = [879]
store_type_879=['d']
assortment_879=['a']
mask_879 = df[(~df['Store'].isin(stores_to_exclude)) & (df['StoreType'].isin(store_type_879)) & (df['Assortment'].isin(assortment_879))]

In [30]:
print (mask_879.shape)
print (mask_879.info())
mask_879.head()

(112826, 18)
<class 'pandas.core.frame.DataFrame'>
Index: 112826 entries, 12 to 1017197
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Store                      112826 non-null  int64         
 1   DayOfWeek                  112826 non-null  int64         
 2   Date                       112826 non-null  datetime64[ns]
 3   Sales                      112826 non-null  int64         
 4   Customers                  112826 non-null  int64         
 5   Open                       112826 non-null  int64         
 6   Promo                      112826 non-null  int64         
 7   StateHoliday               112826 non-null  object        
 8   SchoolHoliday              112826 non-null  int64         
 9   StoreType                  112826 non-null  object        
 10  Assortment                 112826 non-null  object        
 11  CompetitionDistance        111884 non-null

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
12,13,5,2015-07-31,8821,568,1,1,0,0,d,a,310.0,NaN,NaN,1,45.0,2009.0,"Feb,May,Aug,Nov"
19,20,5,2015-07-31,9593,974,1,1,0,0,d,a,2340.0,5.0,2009.0,1,40.0,2014.0,"Jan,Apr,Jul,Oct"
22,23,5,2015-07-31,7273,560,1,1,0,1,d,a,4060.0,8.0,2005.0,0,NaN,NaN,NaN
25,26,5,2015-07-31,7818,611,1,1,0,1,d,a,2300.0,NaN,NaN,0,NaN,NaN,NaN
37,38,5,2015-07-31,7434,608,1,1,0,1,d,a,1090.0,4.0,2007.0,0,NaN,NaN,NaN


In [24]:
print (mask_622.shape)
print (mask_622.info())
mask_622.head()

(204296, 18)
<class 'pandas.core.frame.DataFrame'>
Index: 204296 entries, 6 to 1017207
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Store                      204296 non-null  int64         
 1   DayOfWeek                  204296 non-null  int64         
 2   Date                       204296 non-null  datetime64[ns]
 3   Sales                      204296 non-null  int64         
 4   Customers                  204296 non-null  int64         
 5   Open                       204296 non-null  int64         
 6   Promo                      204296 non-null  int64         
 7   StateHoliday               204296 non-null  object        
 8   SchoolHoliday              204296 non-null  int64         
 9   StoreType                  204296 non-null  object        
 10  Assortment                 204296 non-null  object        
 11  CompetitionDistance        204296 non-null 

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
6,7,5,2015-07-31,15344,1414,1,1,0,1,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN
8,9,5,2015-07-31,8565,687,1,1,0,1,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN
10,11,5,2015-07-31,10457,1236,1,1,0,1,a,c,960.0,11.0,2011.0,1,1.0,2012.0,"Jan,Apr,Jul,Oct"
11,12,5,2015-07-31,8959,962,1,1,0,1,a,c,1070.0,NaN,NaN,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
15,16,5,2015-07-31,10231,979,1,1,0,1,a,c,3270.0,NaN,NaN,0,NaN,NaN,NaN


In [25]:
print (mask_291.shape)
print (mask_291.info())
mask_291.head()

(111884, 18)
<class 'pandas.core.frame.DataFrame'>
Index: 111884 entries, 12 to 1017197
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Store                      111884 non-null  int64         
 1   DayOfWeek                  111884 non-null  int64         
 2   Date                       111884 non-null  datetime64[ns]
 3   Sales                      111884 non-null  int64         
 4   Customers                  111884 non-null  int64         
 5   Open                       111884 non-null  int64         
 6   Promo                      111884 non-null  int64         
 7   StateHoliday               111884 non-null  object        
 8   SchoolHoliday              111884 non-null  int64         
 9   StoreType                  111884 non-null  object        
 10  Assortment                 111884 non-null  object        
 11  CompetitionDistance        111884 non-null

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
12,13,5,2015-07-31,8821,568,1,1,0,0,d,a,310.0,NaN,NaN,1,45.0,2009.0,"Feb,May,Aug,Nov"
19,20,5,2015-07-31,9593,974,1,1,0,0,d,a,2340.0,5.0,2009.0,1,40.0,2014.0,"Jan,Apr,Jul,Oct"
22,23,5,2015-07-31,7273,560,1,1,0,1,d,a,4060.0,8.0,2005.0,0,NaN,NaN,NaN
25,26,5,2015-07-31,7818,611,1,1,0,1,d,a,2300.0,NaN,NaN,0,NaN,NaN,NaN
37,38,5,2015-07-31,7434,608,1,1,0,1,d,a,1090.0,4.0,2007.0,0,NaN,NaN,NaN


In [31]:
# Summary statistics
from scipy.stats import skew, kurtosis
print("Original Data:")
print("Mean:", mask_879['CompetitionDistance'].mean())
print("Median:", mask_879['CompetitionDistance'].median())
print("Skewness:", skew(mask_879['CompetitionDistance']))
print("Kurtosis:", kurtosis(mask_879['CompetitionDistance']))

Original Data:
Mean: 6709.845196810983
Median: 4400.0
Skewness: nan
Kurtosis: nan


In [27]:
# Summary statistics
from scipy.stats import skew, kurtosis
print("Original Data:")
print("Mean:", mask_291['CompetitionDistance'].mean())
print("Median:", mask_291['CompetitionDistance'].median())
print("Skewness:", skew(mask_291['CompetitionDistance']))
print("Kurtosis:", kurtosis(mask_291['CompetitionDistance']))

Original Data:
Mean: 6709.845196810983
Median: 4400.0
Skewness: 2.305205118495136
Kurtosis: 8.81208866389804


In [28]:
# Summary statistics
print("Original Data:")
print("Mean:", mask_622['CompetitionDistance'].mean())
print("Median:", mask_622['CompetitionDistance'].median())
print("Skewness:", skew(mask_622['CompetitionDistance']))
print("Kurtosis:", kurtosis(mask_622['CompetitionDistance']))

Original Data:
Mean: 7022.008752006892
Median: 2330.0
Skewness: 2.806586270422468
Kurtosis: 10.449631701164407


#### Now fill the missing value as CompetitionDistance for the Store 291 

In [35]:
# Fill missing values in 'CompetitionDistance' based on median values as calculated above
df.loc[(df['Store'] == 291) & (df['CompetitionDistance'].isna()), 'CompetitionDistance'] = 4400
df.loc[(df['Store'] == 622) & (df['CompetitionDistance'].isna()), 'CompetitionDistance'] = 2330
df.loc[(df['Store'] == 879) & (df['CompetitionDistance'].isna()), 'CompetitionDistance'] = 4400
df.isna().sum()

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
StoreType                         0
Assortment                        0
CompetitionDistance               0
CompetitionOpenSinceMonth    323348
CompetitionOpenSinceYear     323348
Promo2                            0
Promo2SinceWeek              508031
Promo2SinceYear              508031
PromoInterval                508031
dtype: int64

### Handling missing value for the feature *Promo2SinceWeek*, *Promo2SinceYear* and *PromoInterval*


**Looks like there is a relationship between these columns and lets dig deeper into this**

In [39]:
mask_columns = ['Store','Promo2','Promo2SinceWeek','Promo2SinceYear','PromoInterval']
mask = df[mask_columns]
mask.head()

,Store,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,0,NaN,NaN,NaN
1,2,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,0,NaN,NaN,NaN
4,5,0,NaN,NaN,NaN


In [55]:
mask[(mask['Promo2SinceWeek'] == 1.0) & (df['Promo2'] == 1 ) & (df['Promo2SinceYear'].notnull()) & (df['PromoInterval'].notnull()) ]

,Store,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
10,11,1,1.0,2012.0,"Jan,Apr,Jul,Oct"
34,35,1,1.0,2012.0,"Jan,Apr,Jul,Oct"
60,61,1,1.0,2012.0,"Jan,Apr,Jul,Oct"
64,65,1,1.0,2012.0,"Jan,Apr,Jul,Oct"
97,98,1,1.0,2012.0,"Jan,Apr,Jul,Oct"
...,...,...,...,...,...
1017077,983,1,1.0,2014.0,"Jan,Apr,Jul,Oct"
1017078,984,1,1.0,2013.0,"Jan,Apr,Jul,Oct"
1017087,994,1,1.0,2012.0,"Jan,Apr,Jul,Oct"
1017144,1051,1,1.0,2012.0,"Jan,Apr,Jul,Oct"


In [49]:
print (mask['Promo2SinceWeek'].min())
print (mask['Promo2SinceWeek'].max())

1.0
50.0


In [64]:
mask['Promo2SinceWeek'].describe()

count    509178.000000
mean         23.269093
std          14.095973
min           1.000000
25%          13.000000
50%          22.000000
75%          37.000000
max          50.000000
Name: Promo2SinceWeek, dtype: float64

In [51]:
mask['Promo2SinceWeek'].median()

22.0

**We can notice that if *Promo2* value is 0, *Promo2SinceWeek* has missing value.**

**Intuitively, its correct to have null value in those 3 fearures because if *Promo* is not applied, then monthwise, yearwise and internval is expected null value only**

**Let's fill the minimum value in those NULL values because its MAR and there is relation between these columns. Filling minimum will be appropriate**

In [60]:
# Calculate the minimum value of Promo2SinceWeek, ignoring NaNs
min_promo2_since_week = df['Promo2SinceWeek'].min()

# Fill missing values in Promo2SinceWeek with the minimum value where Promo2 is 0
df.loc[(df['Promo2'] == 0) & (df['Promo2SinceWeek'].isna()), 'Promo2SinceWeek'] = min_promo2_since_week

**We can notice that if *Promo2* value is 0, *Promo2SinceYear* has missing value.**

**Intuitively, its correct to have null value in this fearure because if *Promo2* is not applied, then monthwise, yearwise and internval is expected null value only**

**Let's fill the max value of *Promo2SinceYear* in those NULL values because its MAR and there is relation between these columns. Filling maximum will be appropriate**

In [65]:
print (mask['Promo2SinceYear'].min())
print (mask['Promo2SinceYear'].max())

2009.0
2015.0


In [63]:
# Calculate the maximum value of Promo2SinceYear, ignoring NaNs
max_promo2_since_year = df['Promo2SinceYear'].max()

# Fill missing values in Promo2SinceYear with the maximum year where Promo2 is 0 and Promo2SinceWeek was filled with 1
df.loc[(df['Promo2'] == 0) & (df['Promo2SinceYear'].isna()), 'Promo2SinceYear'] = max_promo2_since_year


**We can notice that if *Promo2* value is 0, *Promo2Interval* has missing value.**

**Intuitively, its correct to have null value in this fearure because if *Promo2* is not applied, then monthwise, yearwise and internval is expected null value only**

**Also, we choose *Promo2SinceWeek* and *Promo2SinceYear* as min and max respectively, it happened for me to calculate what was the most used Promo2Interval for Promo2SinceYear when its max value, so that its appropriate to apply this value.**

In [77]:
df['PromoInterval'].value_counts()

PromoInterval
Jan,Apr,Jul,Oct     293122
Feb,May,Aug,Nov     118596
Mar,Jun,Sept,Dec     97460
Name: count, dtype: int64

In [78]:
# Calculate the maximum value of Promo2SinceYear, ignoring NaNs
max_promo2_since_year = df['Promo2SinceYear'].max()

# Filter the DataFrame where Promo2SinceYear is the maximum value
filtered_df = df[df['Promo2SinceYear'] == max_promo2_since_year]

# Get the count of each unique value in 'PromoInterval' column for the filtered DataFrame
promo_interval_counts filtered_df['PromoInterval'].value_counts()


PromoInterval
Jan,Apr,Jul,Oct     3768
Mar,Jun,Sept,Dec    3400
Feb,May,Aug,Nov     1884
Name: count, dtype: int64

In [79]:
# Fill missing values in Promo2SinceWeek with the minimum value where Promo2 is 0
df.loc[(df['Promo2'] == 0) & (df['PromoInterval'].isna()), 'PromoInterval'] = "Jan,Apr,Jul,Oct"

In [81]:
df.isna().sum()

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
StoreType                         0
Assortment                        0
CompetitionDistance               0
CompetitionOpenSinceMonth    323348
CompetitionOpenSinceYear     323348
Promo2                            0
Promo2SinceWeek                   0
Promo2SinceYear                   0
PromoInterval                     0
dtype: int64